In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import matplotlib.pyplot as plt
import math
import ast
import time

# Configuração de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Semente para reprodutibilidade
torch.manual_seed(42)
np.random.seed(42)

Usando dispositivo: cpu


In [ ]:
train = pd.read_csv('..\\te-aprendizado-de-maquina\\train.csv')
test = pd.read_csv('..\\te-aprendizado-de-maquina\\test.csv')


In [ ]:
# --- Função de Distância Real (Haversine) ---
def haversine_distance(lat1, lon1, lat2, lon2):
    """Calcula a distância em KM entre coordenadas."""
    R = 6371  # Raio da Terra em km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

# --- Função de Ruído GPS (Baseado em Erdmann et al.) ---
def apply_gps_noise(lats, lons, noise_meters=15.0, prob=1.0):
    """
    Adiciona ruído gaussiano às trajetórias.
    noise_meters: desvio padrão do erro em metros.
    """
    # Chance de não aplicar ruído (mantém original)
    if np.random.rand() > prob: return lats, lons
    
    n = len(lats)
    # Gera ruído em metros
    noise_lat = np.random.normal(0, noise_meters, n)
    noise_lon = np.random.normal(0, noise_meters, n)
    
    # Converte metros para graus
    delta_lat = noise_lat / 111111.0
    avg_lat = np.radians(np.mean(lats))
    delta_lon = noise_lon / (111111.0 * np.cos(avg_lat))
    
    # Soma e retorna
    return (np.array(lats) + delta_lat).tolist(), (np.array(lons) + delta_lon).tolist()

In [ ]:
def balance_and_augment_dataset(df, noise_func, n_clusters=3):
    """
    Equilibra o dataset de TREINO gerando cópias ruidosas dos clusters menores.
    """
    print("--- Iniciando Balanceamento e Data Augmentation ---")
    
    # 1. Clusterização rápida baseada no ponto inicial
    points = np.array([[row['path_lat'][0], row['path_lon'][0]] for _, row in df.iterrows()])
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    
    # Trabalhamos numa cópia para não bagunçar o original
    df_temp = df.copy()
    df_temp['cluster_id'] = kmeans.fit_predict(points)
    
    # 2. Identificar o tamanho alvo (tamanho do maior cluster)
    counts = df_temp['cluster_id'].value_counts()
    max_size = counts.max()
    print(f"Contagens Originais:\n{counts}")
    print(f"Meta: {max_size} amostras por cluster.")
    
    dfs_augmented = []
    
    # 3. Augmentation por Cluster
    for cluster_id in range(n_clusters):
        df_c = df_temp[df_temp['cluster_id'] == cluster_id]
        
        # Adiciona os originais
        dfs_augmented.append(df_c) 
        
        # Calcula quantos faltam
        n_needed = max_size - len(df_c)
        
        if n_needed > 0:
            print(f"   -> Cluster {cluster_id}: Gerando {n_needed} cópias sintéticas...")
            # Sorteia amostras existentes para duplicar
            samples = resample(df_c, n_samples=n_needed, random_state=42)
            
            new_rows = []
            for _, row in samples.iterrows():
                nr = row.copy()
                # APLICA O RUÍDO NA CÓPIA
                nr['path_lat'], nr['path_lon'] = noise_func(nr['path_lat'], nr['path_lon'])
                new_rows.append(nr)
            
            dfs_augmented.append(pd.DataFrame(new_rows))
            
    # 4. Finalização
    df_final = pd.concat(dfs_augmented).sample(frac=1, random_state=42).reset_index(drop=True)
    df_final = df_final.drop(columns=['cluster_id'])
    
    print(f"Tamanho Final do Dataset de Treino: {len(df_final)}")
    return df_final

In [ ]:
class GlobalGridDiscretizer:
    def __init__(self, cell_size_meters=200, n_clusters=3):
        self.cell_size = cell_size_meters
        self.n_clusters = n_clusters
        self.grids = {}
        self.token_mapper = LabelEncoder()

    def fit(self, df):
        # Clusterização para definir grades locais
        points = np.array([[r['path_lat'][0], r['path_lon'][0]] for _, r in df.iterrows()])
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=42, n_init=10).fit(points)
        labels = kmeans.labels_
        
        global_offset = 1
        for lbl in set(labels):
            c_points = points[labels == lbl]
            # Define Bounding Box com margem
            lat_min, lon_min = c_points.min(axis=0) - 0.05
            lat_max, lon_max = c_points.max(axis=0) + 0.05
            
            # Calcula passo em graus
            lat_step = self.cell_size / 111111.0
            lon_step = self.cell_size / (111111.0 * np.cos(np.radians((lat_min+lat_max)/2)))
            
            n_rows = math.ceil((lat_max - lat_min) / lat_step)
            n_cols = math.ceil((lon_max - lon_min) / lon_step)
            
            self.grids[lbl] = {'lat_min':lat_min, 'lon_min':lon_min, 
                               'lat_step':lat_step, 'lon_step':lon_step, 
                               'n_rows':n_rows, 'n_cols':n_cols, 'offset':global_offset}
            
            global_offset += (n_rows * n_cols)
            
        # Mapeamento de Tokens (Compactação de IDs)
        print("   Criando vocabulário de tokens...")
        all_tokens = []
        for _, r in df.iterrows():
            all_tokens.extend(self._traj_to_raw(r['path_lat'], r['path_lon']))
            
        self.token_mapper.fit(np.unique(all_tokens))
        self.vocab_size = len(self.token_mapper.classes_) + 1
        print(f"   Vocabulário Final: {self.vocab_size} células únicas.")

    def _traj_to_raw(self, lats, lons):
        tokens = []
        last = None
        for lat, lon in zip(lats, lons):
            t = 0
            # Procura em qual grid o ponto cai
            for grid in self.grids.values():
                if grid['lat_min'] <= lat < (grid['lat_min'] + grid['n_rows']*grid['lat_step']) and \
                   grid['lon_min'] <= lon < (grid['lon_min'] + grid['n_cols']*grid['lon_step']):
                    
                    r = int((lat - grid['lat_min']) / grid['lat_step'])
                    c = int((lon - grid['lon_min']) / grid['lon_step'])
                    t = grid['offset'] + (r * grid['n_cols'] + c)
                    break
            
            # Remove zeros e repetições
            if t != 0 and t != last:
                tokens.append(t)
                last = t
        return tokens

    def transform(self, lats, lons):
        raw = self._traj_to_raw(lats, lons)
        valid = [t for t in raw if t in self.token_mapper.classes_]
        # Retorna tokens mapeados (+1 pois 0 é PAD)
        return (self.token_mapper.transform(valid) + 1).tolist() if valid else [0]

class TrajectoryDataset(Dataset):
    def __init__(self, df, discretizer, scaler, augment=False, mode='train'):
        self.df = df.reset_index(drop=True)
        self.discretizer = discretizer
        self.scaler = scaler
        self.augment = augment
        self.mode = mode # 'train', 'val' ou 'test'

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        lats, lons = row['path_lat'], row['path_lon']
        
        # Augmentation on-the-fly (opcional, se quiser ainda mais variação)
        if self.augment: 
            lats, lons = apply_gps_noise(lats, lons)
            
        tokens = self.discretizer.transform(lats, lons)
        
        # Se for teste, retorna target falso (0.0)
        if self.mode == 'test':
            return torch.tensor(tokens, dtype=torch.long), torch.tensor([0.0, 0.0], dtype=torch.float32)
        else:
            dest = self.scaler.transform([[row['dest_lat'], row['dest_lon']]]).flatten()
            return torch.tensor(tokens, dtype=torch.long), torch.tensor(dest, dtype=torch.float32)

def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs_pad = pad_sequence(inputs, batch_first=True, padding_value=0)
    targets_stack = torch.stack(targets)
    return inputs_pad, targets_stack

In [ ]:
class TrajectoryEncoderAttention(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_dim=128):
        super().__init__()
        # 1. Embedding
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # 2. LSTM Encoder
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        
        # 3. Atenção
        self.attn = nn.Linear(hidden_dim, 1)
        
        # 4. Regressor (Lat, Lon)
        self.regressor = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        # Máscara para ignorar padding
        mask = (x != 0).unsqueeze(-1).float()
        
        embed = self.embedding(x)
        out, _ = self.lstm(embed) # out: [batch, seq, hidden]
        
        # Cálculo dos pesos de atenção
        scores = self.attn(out).masked_fill(mask == 0, -1e9)
        weights = torch.softmax(scores, dim=1)
        
        # Vetor de Contexto (Embedding da Trajetória)
        context = torch.sum(weights * out, dim=1)
        
        # Predição de Coordenadas
        coords = self.regressor(context)
        
        return coords, context

In [ ]:
# 1. Correção de Strings para Listas (se necessário)
def fix_cols(df):
    for c in ['path_lat', 'path_lon']:
        if isinstance(df[c].iloc[0], str):
            df[c] = df[c].apply(ast.literal_eval)
    return df

# Assumindo que 'train' e 'test' já foram carregados do CSV/Pickle
train = fix_cols(train)
if 'test' in globals(): test = fix_cols(test)

# 2. Split Treino/Validação (ANTES do Augmentation)
# Isso garante que a validação seja honesta (apenas dados originais)
train_raw, val_raw = train_test_split(train, test_size=0.2, random_state=42)

# 3. Aplicação do DATA AUGMENTATION (Balanceamento)
# Apenas no conjunto de treino raw
train_aug = balance_and_augment_dataset(train_raw, apply_gps_noise)

# 4. Treinamento do Discretizador e Scaler
# Usamos o train_aug para garantir que o vocabulário cubra as variações ruidosas
discretizer = GlobalGridDiscretizer(cell_size_meters=200)
discretizer.fit(train_aug) 

target_scaler = MinMaxScaler(feature_range=(-1, 1))
target_scaler.fit(train_aug[['dest_lat', 'dest_lon']].values)

# 5. Criação dos DataLoaders
# ds_train usa augmentation=True para variar ainda mais durante as épocas
ds_train = TrajectoryDataset(train_aug, discretizer, target_scaler, augment=True, mode='train')
ds_val   = TrajectoryDataset(val_raw, discretizer, target_scaler, augment=False, mode='val')

dl_train = DataLoader(ds_train, batch_size=64, shuffle=True, collate_fn=collate_fn)
dl_val   = DataLoader(ds_val, batch_size=64, shuffle=False, collate_fn=collate_fn)

print("Setup concluído!")

In [ ]:
# Instancia o Modelo
model = TrajectoryEncoderAttention(
    vocab_size=discretizer.vocab_size,
    embedding_dim=64,
    hidden_dim=128
).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

print("\nIniciando Treinamento...")
train_losses, val_losses = [], []

epochs = 15 
for epoch in range(epochs):
    # --- TREINO ---
    model.train()
    total_loss = 0
    for x, y in dl_train:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        
        pred, _ = model(x) # Ignora contexto aqui
        loss = criterion(pred, y)
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_train = total_loss / len(dl_train)
    train_losses.append(avg_train)
    
    # --- VALIDAÇÃO ---
    model.eval()
    total_val = 0
    with torch.no_grad():
        for x, y in dl_val:
            x, y = x.to(device), y.to(device)
            pred, _ = model(x)
            loss = criterion(pred, y)
            total_val += loss.item()
            
    avg_val = total_val / len(dl_val)
    val_losses.append(avg_val)
    
    print(f"Epoch {epoch+1:02d} | Train Loss: {avg_train:.5f} | Val Loss: {avg_val:.5f}")

In [ ]:
# 1. Gráfico da Função de Perda
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Treino (Com Augmentation)', color='blue')
plt.plot(val_losses, label='Validação (Dados Originais)', color='orange')
plt.title('Curva de Aprendizado')
plt.xlabel('Época')
plt.ylabel('MSE Loss (Normalizada)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 2. Cálculo do Erro Real (km) na Validação
print("\nCalculando métricas reais de distância...")
model.eval()
all_preds, all_targets = [], []

with torch.no_grad():
    for x, y in dl_val:
        x = x.to(device)
        pred, _ = model(x)
        all_preds.append(pred.cpu().numpy())
        all_targets.append(y.numpy())

# Desnormalizar
preds_real = target_scaler.inverse_transform(np.vstack(all_preds))
targs_real = target_scaler.inverse_transform(np.vstack(all_targets))

# Haversine
errors_km = haversine_distance(preds_real[:,0], preds_real[:,1], 
                               targs_real[:,0], targs_real[:,1])

mean_error = np.mean(errors_km)
median_error = np.median(errors_km)

print(f"Erro Médio (MAE):   {mean_error:.3f} km")
print(f"Erro Mediano:       {median_error:.3f} km")

# Histograma
plt.figure(figsize=(10, 4))
plt.hist(errors_km, bins=50, color='purple', alpha=0.7)
plt.title("Distribuição dos Erros em KM")
plt.show()

In [ ]:
if 'test' in globals():
    print("\nGerando predições para submissão...")
    
    # Cria Dataset de Teste (mode='test')
    ds_test = TrajectoryDataset(test, discretizer, target_scaler, augment=False, mode='test')
    dl_test = DataLoader(ds_test, batch_size=64, shuffle=False, collate_fn=collate_fn)
    
    test_preds = []
    
    model.eval()
    with torch.no_grad():
        for x, _ in dl_test:
            x = x.to(device)
            pred, _ = model(x)
            test_preds.append(pred.cpu().numpy())
            
    # Desnormaliza para Lat/Lon reais
    test_preds_real = target_scaler.inverse_transform(np.vstack(test_preds))
    
    # Cria DataFrame
    # Ajuste 'id' conforme a coluna de ID do seu teste
    submission = pd.DataFrame({
        'id': test.index, 
        'lat_pred': test_preds_real[:, 0],
        'lon_pred': test_preds_real[:, 1]
    })
    
    submission.to_csv("submission_kaggle.csv", index=False)
    print("Arquivo 'submission_kaggle.csv' salvo com sucesso!")